<a href="https://colab.research.google.com/github/RockhoRockho/Deep_Learning_Tensorflow/blob/main/20211203_12_%EC%87%BC%ED%95%91%EB%AA%B0_%EA%B3%A0%EA%B0%9D_%EC%A3%BC%EB%AC%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 쇼핑몰 고객 주문 데이터 파악하기**

* 현재 데이터로 부터 상황 파악
* 모델 수립 또는 목표 설정 

### **1-1. 데이터셋**
* 온라인 리테일 사이트의 2010/12 ~ 2011/12까지의 주문 기록 데이터
* 약 500,000건 데이터

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
# pandas를 통해 OnlineRetail.csv 읽어오기
retail = pd.read_csv('./drive/MyDrive/OnlineRetail.csv')

In [ ]:
retail.columns

* InvoiceNo: 주문번호
* StockCode: 아이템 아이디
* Description: 상품 설명
* Quantity: 상품 주문 수량
* InvoiceDate: 주문 시각
* UnitPrice: 상품 가격(동일한 통화)
* CustomerID: 고객 아이디
* Country: 고객 거주 지역(국가)

In [ ]:
retail.head()

In [ ]:
retail.info()

In [ ]:
retail.describe()

In [ ]:
# 각 필드당 Null이 몇개 있는지 확인
retail.isnull().sum()

In [ ]:
# CustomerID의 Null을 제거
retail = retail[pd.notnull(retail['CustomerID'])]
len(retail)

In [ ]:
# 수량, 가격이 0 보다 큰 데이터만 저장( > 0)
retail = retail[retail['Quantity'] > 0]
retail = retail[retail['UnitPrice'] > 0]
len(retail)

In [ ]:
retail.info()

In [ ]:
retail['CustomerID'] = retail['CustomerID'].astype(np.int32)
retail.info()

### **새로운 컬럼 추가**
* 수량 * 가격은 고객의 총 지출비용(CheckoutPrice)

In [ ]:
retail['CheckoutPrice'] = retail['UnitPrice'] * retail['Quantity']
retail.head()

### **정제 데이터 저장**

In [ ]:
# OnlineRetailClean.csv 저장
retail.to_csv('./OnlineRetailClean.csv')

### **1-2. 매출, 가장 많이 팔린 아이템 확인하기**

In [ ]:
import seaborn as sns
COLORS = sns.color_palette()
%matplotlib inline

In [ ]:
dtypes = {
    'UnitPrice': np.float32,
    'Customer ID': np.int32,
    'Quantity': np.int32
}
retail = pd.read_csv('./OnlineRetailClean.csv', dtype=dtypes)
retail.head()  

In [ ]:
retail.info()

### **날짜 타입 데이터 변환**

* 문자열로 데이터를 로딩하는 것보다 data/datatime 타입으로 로딩하는 것이 분석 및 계산에 용이

In [ ]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'], infer_datetime_format=True)

In [ ]:
retail.info()

In [ ]:
retail.head()

In [ ]:
# 전체 매출
total_revenue = retail['CheckoutPrice'].sum()
total_revenue

In [ ]:
# 국가별 매출
rev_by_countries = retail.groupby('Country').sum()['CheckoutPrice'].sort_values()
rev_by_countries

In [ ]:
plot = rev_by_countries.plot(kind='bar', color=COLORS[-1], figsize=(20, 10))
plot.set_xlabel('Country', fontsize=11)
plot.set_ylabel('Revenue', fontsize=11)
plot.set_title('Revenue by Country', fontsize=13)
plot.set_xticklabels(labels=rev_by_countries.index, rotation=45)

In [ ]:
rev_by_countries / total_revenue

In [ ]:
retail['InvoiceDate'].sort_values(ascending=False)

In [ ]:
def extract_month(date): # 201109
  month = str(date.month)
  if date.month < 10:
    month = '0' + month
  return str(date.year) + month

In [ ]:
rev_by_month = retail.set_index('InvoiceDate').groupby(extract_month).sum()['CheckoutPrice']
rev_by_month

def plot_bar(df, xlabel, ylabel, title, color=COLORS[0], figsize=(20, 10), rotation=45):
  plot = df.plot(kind='bar', color=color, figsize=(20, 10))
  plot.set_xlabel(xlabel, fontsize=11)
  plot.set_ylabel(ylabel, fontsize=11)
  plot.set_title(title, fontsize=13)
  plot.set_xticklabels(labels=df.index, rotation=rotation)

plot_bar(rev_by_month, 'Month', 'Revenue', 'Revenue by Month')

**요일별 매출**

In [ ]:
from datetime import datetime

In [ ]:
retail['Daytime'] = retail['InvoiceDate'].dt.day_name()
retail.groupby('Daytime').sum()['CheckoutPrice'].sort_values()

In [ ]:
rev_by_dow = retail.set_index('InvoiceDate').groupby(lambda date:date.dayofweek).sum()['CheckoutPrice']
rev_by_dow

In [ ]:
DAY_OF_WEEK = np.array(['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun'])
rev_by_dow.index = DAY_OF_WEEK[rev_by_dow.index]
plot_bar(rev_by_dow, 'DOW', 'REVENUE', 'REVENUE by DOW')

**시간별 매출**

In [ ]:
rev_by_hour = retail.set_index('InvoiceDate').groupby(lambda date:date.hour).sum()['CheckoutPrice']
rev_by_hour
plot_bar(rev_by_hour, 'hour', 'Revenue', 'Revenue by hour')

### **매출 데이터로부터 insight**
* 전체 매출의 82%가 UK에서 발생
* 11년도의 가장 많은 주문이 발생한 달은 11월(12월의 전체 데이터가 반영되지 않음)
* 11월의 판매량이 압도적으로 많음
* 일주일 중 목요일까지는 성장세, 이후로 하락(토요일에는 주문을 받지 않음)
* 7시에 주문 시작, 12시까지 증가세, 15시까지 하락, 15시 이후부터 급락

### **제품별 metrics**

In [ ]:
# Top 10 판매 제품
top_selling = retail.groupby('StockCode').sum()['Quantity'].sort_values(ascending=False)[:10]
top_selling

In [ ]:
# Top 10 매출 제품
top_revenue = retail.groupby('StockCode').sum()['CheckoutPrice'].sort_values(ascending=False)[:10]

### **Top3 아이템의 월별 판매량 추이**

In [ ]:
monthly_top3 = retail.set_index('InvoiceDate').groupby(['StockCode', extract_month]).sum()[['Quantity', 'CheckoutPrice']].loc[top_selling.index[:3]]
monthly_top3

In [ ]:
plot_bar(monthly_top3['CheckoutPrice'], 'Product/Month', 'Revenue', 'Revenue of top3 items')